In [ ]:
import os
import pickle
from typing import List

import numpy as np
import open3d as o3d
import trimesh

from realtime_trees.clustering import cluster
from realtime_trees.terrain import fit_terrain
from realtime_trees.tree import Tree
from realtime_trees.tree_manager import TreeManager
from realtime_trees.utils.meshing import meshgrid_to_mesh
from realtime_trees.utils.ros_proxys import Time


pcd_file = "/path/to/your/cloud.pcd"
tree_manager = TreeManager()

debug_level = 0
max_tree_radius = 0.5 # set accoring to trees in the input cloud for compute efficiency
cloud = o3d.io.read_point_cloud(pcd_file, print_progress=False)
# compute normals if not already present
if not cloud.has_normals():
    cloud.estimate_normals()
cloud = o3d.t.geometry.PointCloud.from_legacy(cloud)

# FIT TERRAIN
terrain = fit_terrain(cloud=cloud, sloop_smooth=False, cloth_cell_size=2.0)
terrain_verts, terrain_tris = meshgrid_to_mesh(terrain)

# SEGMENT TREES
clusters = cluster(
    cloud=cloud,
    terrain=terrain,
    crop_bounds=[[0.5, 1.25], [1.5, 2.25], [2.5, 3.25], [3.5, 4.25], [4.5, 5.25]],
    # crop_bounds=[[0.5, 1.5], [2., 3.], [4., 5.]],
    max_cluster_radius=5.0,
    max_tree_radius = max_tree_radius,
    debug_level=debug_level,
    dbscan_eps=0.2
)
print(f"found {len(clusters)} clusters")

# FIT TREES
trees: List[Tree] = []
for c in clusters:
    tree = Tree(id=c.info.id)
    c.info.T_sensor2map = np.eye(4)
    c.info.time_stamp = Time(0, 0)
    tree.add_cluster(c)
    tree.reconstruct(
        max_height = None, # Max height of reconstruction. If None, max height of points is used
        slice_heights = 1.0, # Either a float for the increment or an iterable of slice heights
        slice_thickness = 0.1, # Thickness of a slice
        max_center_deviation = 0.3, # maximum deviation of a circle with respect to the circle beneath it
        max_radius = max_tree_radius, # maximum radius of tree
        filter_radius = 0.1, # Distance used for filtering to consider a point represented by a circle model
        max_consecutive_fails = 5, # stopping criterion for maximum consecutive failed attempts along the tree trunk
        min_tree_height = 3,
    )
    trees.append(tree)
print(f"found {len(trees)} trees from which {len([t for t in trees if t.reconstructed])} are reconstructed")

# PLOT TREE MESHES
viz_objects = [cloud.to_legacy(),]
for tree in trees:
    verts, tris = tree.generate_mesh()
    if verts is None or tris is None:
        continue

    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(verts),
        o3d.utility.Vector3iVector(tris)
    )
    mesh.compute_vertex_normals()
    viz_objects.append(mesh)

    viz_objects.append(tree.clusters[0].cloud.to_legacy())
o3d.visualization.draw_geometries(viz_objects)

# EXPORT TREE MANAGER
tree_manager.trees = trees
tree_manager.save_as_zip(os.path.join(os.path.dirname(pcd_file), "tree_manager.zip"))
print("Done!")